# 05 Recommender function:

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm
tqdm.pandas()
from openai import OpenAI

import os
import re
import ast
import random
import numpy as np

from langchain_openai import OpenAIEmbeddings
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
pd.set_option('display.max_columns',None)

In [ ]:
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large",
    api_key='')

<h3>Importing vectorized databases and instatiating Faiss model</h3>

In [5]:
VS_content_based = FAISS.load_local(
    "../data/04_vector_store/VS_collab_based", embeddings, allow_dangerous_deserialization=True
)

VS_collab_based = FAISS.load_local(
    "../data/04_vector_store/VS_content_based", embeddings, allow_dangerous_deserialization=True
)

In [161]:
#User from validation DS - USER_ID = 994489
purchased_games = [
 'counterstrikeglobaloffensive', 'deathmatchclassic', 'dayofdefeat',
 'cthulhusavestheworld', 'aiwarfleetcommand', 'counterstrike',
 'chernobylcommando', 'breathofdeathvii', 'dota2', 'hackerevolution',
 'halflife', 'halflifeopposingforce', 'huntsmantheorphanagehalloweenedition',
 'raceroomracingexperience', 'ricochet', 'scourgeoutbreak',
 'teamfortressclassic', 'halflifeblueshift'
 ]

New_inference = [
  'counterstrikeglobaloffensive', 'deathmatchclassic', 'dayofdefeat',
  'cthulhusavestheworld', 'aiwarfleetcommand', 'counterstrike',
  'chernobylcommando', 'breathofdeathvii', 'dota2', 'hackerevolution'
]

label = [
 'halflife', 'halflifeopposingforce', 'huntsmantheorphanagehalloweenedition',
 'raceroomracingexperience', 'ricochet', 'scourgeoutbreak',
 'teamfortressclassic', 'halflifeblueshift'
]

#Inference games description
explicative_features_inference="Free to Play,MOBA,Multiplayer,Strategy,e-sports,Team-Based,Competitive,Action,Online Co-Op,PvP,Difficult,Co-op,RTS,Tower Defense,Fantasy,RPG,Character Customization,Replay Value,Action RPG,Simulation|Multi-player,Co-op,Steam Trading Cards,Steam Workshop,SteamVR Collectibles,In-App Purchases,Valve Anti-Cheat enabled|[No descriptionBulgarianNo description, No descriptionCzechNo description, No descriptionDanishNo description, No descriptionDutchNo description, No descriptionEnglishNo description, No descriptionFinnishNo description, No descriptionFrenchNo description, No descriptionGermanNo description, No descriptionGreekNo description, No descriptionHungarianNo description, No descriptionItalianNo description, No descriptionJapaneseNo description, No descriptionKoreanNo description, No descriptionNorwegianNo description, No descriptionPolishNo description, No descriptionPortugueseNo description, No descriptionPortuguese - BrazilNo description, No descriptionRomanianNo description, No descriptionRussianNo description, No descriptionSimplified ChineseNo description, No descriptionSpanish - SpainNo description, No descriptionSwedishNo description, No descriptionThaiNo description, No descriptionTraditional ChineseNo description, No descriptionTurkishNo description, No descriptionUkrainianNo description, No descriptionSpanish - Latin AmericaNo description, No descriptionVietnameseNo description]|150000000.0|0.0 FPS,Shooter,Multiplayer,Competitive,Action,Team-Based,e-sports,Tactical,First-Person,PvP,Online Co-Op,Co-op,Strategy,Military,War,Difficult,Trading,Realistic,Fast-Paced,Moddable|Multi-player,Steam Achievements,Full controller support,Steam Trading Cards,Steam Workshop,In-App Purchases,Valve Anti-Cheat enabled,Stats,Remote Play on Phone,Remote Play on Tablet,Remote Play on TV|[No descriptionCzechNo description, No descriptionDanishNo description, No descriptionDutchNo description, No descriptionEnglishNo description, No descriptionFinnishNo description, No descriptionFrenchNo description, No descriptionGermanNo description, No descriptionHungarianNo description, No descriptionItalianNo description, No descriptionJapaneseNo description, No descriptionKoreanNo description, No descriptionNorwegianNo description, No descriptionPolishNo description, No descriptionPortugueseNo description, No descriptionPortuguese - BrazilNo description, No descriptionRomanianNo description, No descriptionRussianNo description, No descriptionSimplified ChineseNo description, No descriptionSpanish - SpainNo description, No descriptionSwedishNo description, No descriptionThaiNo description, No descriptionTraditional ChineseNo description, No descriptionTurkishNo description, No descriptionBulgarianNo description, No descriptionUkrainianNo description, No descriptionGreekNo description, No descriptionSpanish - Latin AmericaNo description, No descriptionVietnameseNo description]|75000000.0|0.0 Action,FPS,Multiplayer,Shooter,Classic,Team-Based,First-Person,Competitive,Tactical,e-sports,1990No descriptions,PvP,Old School,Military,Strategy,Survival,Score Attack,1980s,Assassin,Nostalgia|Multi-player,PvP,Online PvP,Shared/Split Screen PvP,Valve Anti-Cheat enabled|[No descriptionEnglishNo description, No descriptionFrenchNo description, No descriptionGermanNo description, No descriptionItalianNo description, No descriptionSpanish - SpainNo description, No descriptionSimplified ChineseNo description, No descriptionTraditional ChineseNo description, No descriptionKoreanNo description]|15000000.0|9.99 Simulation,Hacking,Puzzle,Singleplayer,Text-Based,Strategy,Cyberpunk,Difficult,Indie,Action,Education,Exploration,Immersive Sim,Typing,2D,Minimalist,Funny,Tactical,Anime,Free to Play|Single-player,Steam Trading Cards|[No descriptionEnglishNo description]|350000.0|0.54 Action,FPS,Classic,Multiplayer,Shooter,First-Person,Arena Shooter,Old School,Sci-fi,Competitive,Fast-Paced,Retro,Gore,Co-op,Difficult,1990No descriptions|Multi-player,PvP,Online PvP,Shared/Split Screen PvP,Valve Anti-Cheat enabled,Remote Play Together|[No descriptionEnglishNo description, No descriptionFrenchNo description, No descriptionGermanNo description, No descriptionItalianNo description, No descriptionSpanish - SpainNo description, No descriptionKoreanNo description, No descriptionRussianNo description, No descriptionSimplified ChineseNo description, No descriptionTraditional ChineseNo description]|7500000.0|4.99 FPS,World War II,Multiplayer,Shooter,Action,War,Team-Based,Classic,First-Person,Class-Based,Military,Historical,Tactical,Singleplayer,Co-op,Difficult,Old School,Retro,World War I,Strategy|Multi-player,Valve Anti-Cheat enabled|[No descriptionEnglishNo description, No descriptionFrenchNo description, No descriptionGermanNo description, No descriptionItalianNo description, No descriptionSpanish - SpainNo description]|7500000.0|4.99 Strategy,RTS,Space,Indie,Sci-fi,Co-op,4X,Grand Strategy,Difficult,Tactical,Great Soundtrack,Singleplayer,Multiplayer,Tower Defense,Simulation,2D,Replay Value,Real-Time with Pause,Cult Classic,Classic|Single-player,Multi-player,Co-op,Online Co-op,LAN Co-op,Cross-Platform Multiplayer,Steam Achievements,Steam Trading Cards|[No descriptionEnglishNo description]|350000.0|9.99 JRPG,Lovecraftian,Comedy,RPG,Indie,Retro,Turn-Based,Fantasy,Funny,Singleplayer,Adventure,Parody,Pixel Graphics,2D,Great Soundtrack,Dark Humor,Old School,Action,Memes,Story Rich|Single-player,Full controller support,Steam Trading Cards,Remote Play on Phone,Remote Play on Tablet,Remote Play on TV|[No descriptionEnglishNo description]|750000.0|2.99 JRPG,RPG,Funny,Indie,Retro,Comedy,Pixel Graphics,Turn-Based,Singleplayer,Parody,Short,Fantasy,2D,Adventure,Old School,Horror|Single-player,Full controller support|[No descriptionEnglishNo description]|750000.0|2.99"

<h3>Collaborative filtering Model</h3>

In [ ]:
collab_results = VS_collab_based.similarity_search(
  str(New_inference),
  k=10,
  )

New_inference = [item.split(',')[0] for item in New_inference]
colab_base_rec = [result.page_content for result  in collab_results]
full_list=[]
for i in collab_results:
  converted_lists = ast.literal_eval(i.page_content)
  converted_lists = list(set(converted_lists))
  converted_lists = [item.split(',')[0] for item in converted_lists]
  full_list.append(converted_lists)

colab_base_rec = list(set([item for sublist in full_list for item in sublist]))
colab_base_rec = list(set(colab_base_rec)-set(New_inference))

print("Recommendation based on other players with similar gaming taste:\n",colab_base_rec)
print("\n\nLabel - other purchased games from the same user that weren't used on inference:\n",label)
print("\n\nOverlay between Label and recommendation:\n",list(set(colab_base_rec) & set(label)))
print("\n\nAccuracy:\n",len(list(set(colab_base_rec) & set(label)))/len(label))

Recommendation based on other players with similar gaming taste:
 ['secretsofgrindea', 'spiralknights', 'unturned', 'defiance', 'halflife2episodeone', 'left4dead', 'halflife2', 'halflife', 'counterstrikeconditionzero', 'teamfortress2', 'redfactionguerrillasteamedition', 'doubleactionboogaloo', 'halflife2deathmatch', 'portal', 'sniperelite', 'killingfloor', 'hitmanabsolution', 'reignofkings', 'halflifeopposingforce', 'sourcefilmmaker', 'worldofgunsgundisassembly', 'farcry3', 'portal2', 'dayofdefeatsource', 'halflife2episodetwo', 'left4dead2', 'payday2', 'sidmeierscivilizationv', 'garrysmod', 'counterstrikesource', 'dcuniverseonline']


Label - other purchased games from the same user that weren't used on inference:
 ['halflife', 'halflifeopposingforce', 'huntsmantheorphanagehalloweenedition', 'raceroomracingexperience', 'ricochet', 'scourgeoutbreak', 'teamfortressclassic', 'halflifeblueshift']


Overlay between Label and recommendation:
 ['halflifeopposingforce', 'halflife']


Accuracy:

----

<h3>Content based Model</h3>

In [170]:
content_results = VS_content_based.similarity_search(
  str(explicative_features_inference),
  k=30,
  )

content_results_list=[]
for i in content_results:
  content_results_list.append(i.id)

collab_results = list(set(content_results_list)-set(New_inference))
print("Recommendation based on similar games the user engajed:\n",content_results_list)
print("\n\nLabel - other purchased games from the same user that weren't used on inference:\n",label)
print("\n\nOverlay between Label and recommendation:\n",list(set(content_results_list) & set(label)))
print("\n\nAccuracy:\n",len(list(set(content_results_list) & set(label)))/len(label))

Recommendation based on similar games the user engajed:
 ['counterstrikeconditionzero', 'doorkickers', 'teamfortressclassic', 'divinityoriginalsinenhancededition', 'halospartanassault', 'naturalselection2', 'torchlightii', 'borderlands2', 'depth', 'calltoarms', 'dota2', 'sanctum2', 'tomclancysrainbowsixsiege', 'mondaynightcombat', 'mitosisthegame', 'herosiege', 'fableanniversary', 'primalcarnageextinction', 'nationred', 'payday2', 'cronix', 'wargamereddragon', 'ofguardsandthieves', 'primeworld', 'fistfuloffrags', 'killingfloor2', 'left4dead2', 'combatmonsters', 'eyedivinecybermancy', 'dungeondefenders']


Label - other purchased games from the same user that weren't used on inference:
 ['halflife', 'halflifeopposingforce', 'huntsmantheorphanagehalloweenedition', 'raceroomracingexperience', 'ricochet', 'scourgeoutbreak', 'teamfortressclassic', 'halflifeblueshift']


Overlay between Label and recommendation:
 ['teamfortressclassic']


Accuracy:
 0.125


<h3>Hybrid based Model</h3>

In [171]:
hybrid_results = list(set(content_results_list) & set(colab_base_rec))

print("Recommendation based on hybrid aproach (Collaborative & content based):\n",hybrid_results)
print("\n\nLabel - other purchased games from the same user that weren't used on inference:\n",label)
print("\n\nOverlay between Label and recommendation:\n",list(set(hybrid_results) & set(label)))
print("\n\nAccuracy:\n",len(list(set(hybrid_results) & set(label)))/len(label))

Recommendation based on hybrid aproach (Collaborative & content based):
 ['counterstrikeconditionzero', 'left4dead2', 'payday2']


Label - other purchased games from the same user that weren't used on inference:
 ['halflife', 'halflifeopposingforce', 'huntsmantheorphanagehalloweenedition', 'raceroomracingexperience', 'ricochet', 'scourgeoutbreak', 'teamfortressclassic', 'halflifeblueshift']


Overlay between Label and recommendation:
 []


Accuracy:
 0.0


# It is important to remember that even if the recomendation isn't on user games list, it still likelly to purchase